In [1]:
import subprocess


In [3]:
result = subprocess.run(['echo', 'Hello from the child!'],capture_output=True,encoding='utf-8')
result.check_returncode() # No exception means clean exit
print(result.stdout)

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [4]:
proc = subprocess.Popen(['sleep', '1'])
while proc.poll() is None:
    print('Working...')
print('Exit status', proc.poll())

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [5]:
def factorize(number):
    for i in range(1, number + 1):
        if number % i == 0:
            yield i

In [9]:
import time
numbers = [2139079, 1214759, 1516637, 1852285]
start = time.time()
for number in numbers:
    list(factorize(number))
end = time.time()
delta = end - start
print(f'Took {delta:.3f} seconds')

Took 0.256 seconds


In [7]:
from threading import Thread
class FactorizeThread(Thread):
    def __init__(self, number):
        super().__init__()
        self.number = number
    def run(self):
        self.factors = list(factorize(self.number))

In [10]:
start = time.time()
threads = []
for number in numbers:
    thread = FactorizeThread(number)
    thread.start()
    threads.append(thread)
for f in threads:
    f.join()
end = time.time()
delta = end - start
print(f'Took {delta:.3f} seconds')

Took 0.262 seconds


In [11]:
import select
import socket
def slow_systemcall():
    select.select([socket.socket()], [], [], 0.1)

In [12]:
start = time.time()
for _ in range(5):
    slow_systemcall()
end = time.time()
delta = end - start
print(f'Took {delta:.3f} seconds')

Took 0.545 seconds


In [15]:
start = time.time()
threads = []
for _ in range(5):
    thread = Thread(target=slow_systemcall)
    thread.start()
    threads.append(thread)
def compute_helicopter_location(index):
    ...
for i in range(5):
    compute_helicopter_location(i)
for thread in threads:
    thread.join()
end = time.time()
delta = end - start
print(f'Took {delta:.3f} seconds')

Took 0.116 seconds


In [ ]:
Using threads is the simplest way to do blocking I/O in parallel with 
minimal changes to your program.

In [23]:
class Counter:
    def __init__(self):
        self.count = 0
    def increment(self, offset):
        self.count += offset
def worker(sensor_index, how_many, counter):
    for _ in range(how_many):
        counter.increment(1)

In [27]:
from threading import Thread
how_many = 50**5
counter = Counter()
threads = []
for i in range(5):
    thread = Thread(target=worker,
    args=(i, how_many, counter))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
expected = how_many * 5
found = counter.count
print(f'Counter should be {expected}, got {found}')

Counter should be 1562500000, got 826123856


In [25]:
from threading import Lock
class LockingCounter:
    def __init__(self):
        self.lock = Lock()
        self.count = 0
    def increment(self, offset):
        with self.lock:
            self.count += offset

In [26]:
counter = LockingCounter()
for i in range(5):
    thread = Thread(target=worker,
    args=(i, how_many, counter))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
expected = how_many * 5
found = counter.count
print(f'Counter should be {expected}, got {found}')

Counter should be 500000, got 500000


In [ ]:
There are several async stuff, and they are not good. Need to read them if I work on async.